# 载入数据

In [1]:
import pandas as pd
df = pd.read_csv("datascience.csv", encoding='gb18030')

打印数据的头几行

In [2]:
df.head()

,title,author,content
0,大数据产业迎政策暖风 最新大数据概念股一览,财经热点扒客,大数据产业发展受到国家重视，而大数据已经上升为国家战略，未来发展前景很广阔。大数据产业“十三...
1,Google发布机器学习平台Tensorflow游乐场～带你一起玩神经网络！,硅谷周边,点击上方“硅谷周边”关注我，收到最新的文章哦！昨天，Google发布了Tensorflow游...
2,李克强：中国大数据和云计算产业是开放的,苏州高新区金融办,国务院总理李克强当地时间20日上午在纽约下榻饭店同美国经济、金融、智库、媒体等各界人士座谈，...
3,全峰集团持续挖掘大数据,快递物流网,2016年，全峰集团持续挖掘大数据、云计算、“互联网+”等前沿技术和物流快递的融合，并通过优...
4,第366期【微理工】贵州理工学院召开大数据分析与应用专题分享会,贵州理工学院,贵州理工学院召开大数据分析与应用专题分享会 借“创响中国”贵安站巡回接力活动暨2016贵安大...


打印数据框的大小

In [3]:
df.shape

(1024, 3)

数据框一共有1024行，共3列

# 分词

载入结巴分词，并定义分词函数

In [4]:
import jieba
def chinese_word_cut(mytext):
    '''
    分词函数：将分词后的结果串用空格拼接
    '''
    return " ".join(jieba.cut(mytext))

用`pandas`的apply函数对训练语料进行分词

In [5]:
df["content_cutted"] = df.content.apply(chinese_word_cut)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/z9/gmjx7g0x72d_yy6m48nhvdtm0000gn/T/jieba.cache
Loading model cost 0.420 seconds.
Prefix dict has been built succesfully.


执行完毕之后，我们需要查看一下，文本是否已经被正确分词。



In [6]:
df.content_cutted.head()

0    大 数据 产业 发展 受到 国家 重视 ， 而 大 数据 已经 上升 为 国家 战略 ， 未...
1    点击 上方 “ 硅谷 周边 ” 关注 我 ， 收到 最新 的 文章 哦 ！ 昨天 ， Goo...
2    国务院 总理 李克强 当地 时间 20 日 上午 在 纽约 下榻 饭店 同 美国 经济 、 ...
3    2016 年 ， 全峰 集团 持续 挖掘 大 数据 、 云 计算 、 “ 互联网 + ” 等...
4    贵州 理工学院 召开 大 数据分析 与 应用 专题 分享 会   借 “ 创响 中国 ” 贵...
Name: content_cutted, dtype: object

# 用gensim训练主题模型

# 清洗标点、停用词

In [7]:
import re
from nltk.corpus import stopwords

def load_punctuations():
    pun_list = []
    with open('./pun_list.txt') as fr:
        for line in fr:
            line = line.strip().decode('utf8')
            pun_list.append(line)
    return pun_list

# 英文停用词
english_stopwords = stopwords.words('english')
chinese_stopwords = {}.fromkeys([line.rstrip().decode('gbk') for line in open('chinese_stopwords.txt')])
# 标点
pun_list = load_punctuations()
def clean_text(text):
    text = text.strip()
    # 去除标点
    for pun in pun_list:
        text = text.replace(pun, ' ')
    new_text = ' '.join([w for w in text.split() if w not in english_stopwords and w not in chinese_stopwords and len(w)>1])
    return new_text

# 切词

In [8]:
import gensim
from gensim.models.ldamodel import LdaModel
from gensim import corpora
from nltk import wordpunct_tokenize

class Token_Corpus(object):
    def __init__(self,corpus):
        self.corpus = corpus

    def __iter__(self):
        for text in self.corpus:
            text = text.strip()
            text = clean_text(text)
            yield self.tokenize(text)
            
    def tokenize(self, text):
        token = wordpunct_tokenize(text)
        return token

documents = list(df["content_cutted"])
tokenized_corpus = Token_Corpus(documents)

/Users/chenhongsheng/.pyenv/versions/anaconda2-4.3.1/lib/python2.7/site-packages/cffi/model.py:532: UserWarning: 'point_conversion_form_t' has no values explicitly defined; guessing that it is equivalent to 'unsigned int'
  % self._get_c_name())


In [9]:
# 测试文本清洗
doc = documents[1]
print clean_text(doc)

点击 上方 硅谷 周边 关注 收到 最新 文章 昨天 Google 发布 Tensorflow 游乐场 Tensorflow Google 今年 推出 机器 学习 开源 平台 Tensorflow 游乐场 浏览器 训练 神经网络 酷酷 图像 直观 了解 神经网络 工作 原理 今天 就让 硅谷 周边 一起 Tensorflow 游乐场 快乐 玩耍 昨天 Google 深度 学习 部门 Google Brain 掌门人 Google 万众 景仰 级别 工程师 Jeff Dean Google Plus 发布 Tensorflow 游乐场 消息 小伙伴 十分 激动 地去 Tensorflow 网站 神经网络 游乐场 地址 http playground tensorflow org 快点 一起 看看 游乐场 好玩 东东 打开 网站 看见 上面 标语 浏览器 神经网络 不用 担心 不坏 令人振奋 面对 随便 按钮 看看 数据 游乐场 形态 数据 每组 数据 形态 分布 一群 一个点 与生俱来 特征 x1 x2 表示 位置 数据 橙色 蓝色 神经网络 目标 训练 知道 位置 橙色 位置 蓝色 橙色 橙子 蓝色 蓝莓 假设 2000 散落 各处 橙子 蓝莓 1000 知道 坐标 蓝莓 橙子 蓝莓 信息 训练 神经网络 能够 准确 预测出 1000 橙子 蓝莓 上面 数据 发现 相对 简单 地区 分开 最后 一组 螺旋 数据 最难 Tensorflow 游乐场 数据 十分 灵活 调整 noise 干扰 大小 改变 训练 数据 测试数据 比例 下图 noise 数据分布 一个 数据 信息 喂给 机器 学习 系统 需要 feature extraction 特征提取 真的 区分 蓝莓 橙子 大小 颜色 特征 一个点 x1 x2 两个 特征 两个 特征 衍生 许多 特征 抽象 来说 机器 学习 classifier 分类器 试图 一条 多条 能够 100 正确 地区 蓝色 橙色 蓝色 点会 在线 一边 橙色 另一边 上面 直观 第一张 图中 x1 唯一 特征 一条 x1 垂直 改变 参数 条线 左右 移动 特征 容易 看出 需要 智能 结合 一种 多种 特征 能够 成功 蓝色 橙色 分类 feature extraction 往往 机器 学习 应用 最难 部分 神经网络 能够 完成 大

# 生成字典

In [10]:
dictionary = corpora.Dictionary(tokenized_corpus)
# 过滤掉出现次数少于20或在不超过10%的文档中出现的词
dictionary.filter_extremes(no_below=20, no_above=0.1)
print(dictionary)

Dictionary(2402 unique tokens: [u'\u91d1\u878d\u673a\u6784', u'\u5982\u540c', u'\u8fd8\u8981', u'\u7f16\u8bd1', u'\u4e3a\u4e3b']...)


# 生成稀疏矩阵


In [11]:
class MyCorpus(object):
    def __init__(self, token_list, dictionary):
        self.token_list = token_list
        self.dictionary = dictionary
    
    def __iter__(self):
        for tokens in self.token_list:
            yield self.dictionary.doc2bow(tokens)
            
corpus = MyCorpus(tokenized_corpus,dictionary)
%time gensim.corpora.MmCorpus.serialize('data_science.mm', corpus)


mm_corpus = gensim.corpora.MmCorpus('data_science.mm')
print(mm_corpus)

CPU times: user 5.58 s, sys: 82.1 ms, total: 5.66 s
Wall time: 5.89 s
MmCorpus(1024 documents, 2402 features, 103771 non-zero entries)


# 训练LDA模型

In [12]:
num_topics = 20
%time lda_model = LdaModel(mm_corpus, num_topics=num_topics, id2word=dictionary, passes=20)

CPU times: user 2min 34s, sys: 2.06 s, total: 2min 36s
Wall time: 2min 44s


# 预测给定文档的主题概率分布

In [13]:
doc_num = 45

# Returns Topic + Probability of Topic
lda_vector = lda_model[mm_corpus[doc_num]]
# Prints the most likely Topic. Performs Max based on the 2nd element in the tuple
print(max(lda_vector, key=lambda item: item[1]))
print(lda_model.print_topic(max(lda_vector, key=lambda item: item[1])[0]))
print(documents[doc_num])

(4, 0.36161469088467285)
0.037*"信用卡" + 0.035*"案件" + 0.023*"亚马逊" + 0.020*"有限公司" + 0.016*"法律" + 0.013*"物流" + 0.013*"犯罪" + 0.012*"贵州" + 0.012*"申请" + 0.011*"保障"
学 法网 APP ， 法律 人 必备 ！ （ 司考 交流 ， 法律 路上 ， 法律 求职 ） 长 按 识别 二维码 下载 学 法网 APP2016 年 国家 司法考试 已 于 今天 （ 9 月 25 日 ） 落下 帷幕 ， 学 法网 （ xuefa . com ） 迎来 了 访问 高峰 ， 学 法网 APP 的 打开 率 也 急剧 上升 （ 打开 学 法网 缓慢 的 童鞋 ， 请 耐心 等待 一下 哦 ） 。 在 学 法网 APP 上 ， 有吐槽 2016 年司 考真题 的 、 有司 考 难易度 的 投票 、 有 总结 自己 考试 经历 的 等等 。 考虑 到 大家 刚考 完 不想 看太多 文字 ， 于是 本小 编特 将 学法 网上 一些 热门 投票 以 “ 数据 ” 为 核心 ， 并 以 图片 形式 展现 给 大家 ， 以 让 大家 能 更 宏观 的 了解 2016 年 司法考试 。 一 、 2016 年司 考 各卷 难度 排行 ： 卷 四 > 卷 二 > 卷 三 > 卷 一 25 日 下午 考试 结束 以后 ， 学 法网 第一 时间 就 在 网上 对 参加 了 当天 考试 的 考生 进行 了 《 2016 年 司法考试 ， 哪卷 最难 ？ 》 的 投票 ， 被 吐 槽 最 多 的 是 卷 二 和 卷 四 （ 觉得 卷 四 最难 的 占 38.42% ， 觉得 卷二 最难 的 占 34.48% ） 、 其次 是 卷 三 ， 最后 是 卷 一 ， 数据 如下 图 ： 投票 地址 ： http : / / bbs . xuefa . com / thread - 540535 - 1 - 1 . html 二 、 2016 年 司考考 场 缺考 情况 ： 司考 人数 很多 ， 打 酱油 的 也 很多 通过 分析 学法 网上 的 投票 贴 《 2016 年司 考 ， 还记 的 您 的 考场 上 缺考 几人 吗 ？ 》 得出结论 ， 考场 缺考 人数 达到 6 人

# 主题分析

In [14]:
for i in range(num_topics):
    print '主题:',i
    for word, prob in lda_model.show_topic(i, 10):
        print word, prob

主题: 0
检索 0.0527524391955
专利 0.031551076106
信用 0.024616220332
保护 0.0227458187498
评分 0.0172172459105
征信 0.0168344542381
申请 0.012125020056
id 0.0089064639305
文献 0.00889650519672
法律 0.0088057430596
主题: 1
阿里 0.0114336424424
游戏 0.0107874357185
员工 0.00974455163699
付费 0.00927909442845
腾讯 0.00900040162823
新闻 0.00859241639632
顾客 0.00763218486202
大脑 0.00759843992203
受众 0.00644905502718
故事 0.00540966174524
主题: 2
图表 0.0403536379439
新闻 0.0177548841274
回复 0.0158277659964
代理 0.0152968996105
图形 0.0131574869772
地图 0.012467339161
浏览器 0.00859128156752
下载 0.00766914465587
微博 0.00757557739323
网页 0.00701852306951
主题: 3
患者 0.0218494224586
治疗 0.0209191022829
诊断 0.0160351006566
医学 0.0149695743829
距离 0.0148466569617
药物 0.0141210865563
疾病 0.014010191479
医生 0.0132834256622
驾驶 0.0120614467353
员工 0.0119718592747
主题: 4
信用卡 0.0365309684932
案件 0.0346776908406
亚马逊 0.0227576315194
有限公司 0.019814252683
法律 0.0160360607286
物流 0.0132644394252
犯罪 0.0126752855114
贵州 0.01163674693
申请 0.0115850881369
保障 0.0107233227709
主题: 5
教授 0

# 用HDP确定主题数

In [15]:
from gensim import models
# 最大主题数为50
hdp = models.hdpmodel.HdpModel(mm_corpus, dictionary, T=50)


打印主题数

In [16]:
num_hdp_topics = len(hdp.print_topics(num_topics=-1, num_words=20))
print num_hdp_topics

50


In [17]:
for i in range(num_hdp_topics):
    print '主题:',i
    for word, prob in hdp.show_topic(i, topn=20):
        print word, prob


主题: 0
大众 0.00350751711697
人口 0.00288858498672
工程 0.00240553091894
保护 0.00233686650757
资产 0.00215212953802
实验室 0.00187455212937
游戏 0.00184934218615
集团 0.0017692770693
AI 0.00171163931979
感知 0.00165077090976
回复 0.00159472291475
农业 0.00157917020373
亚马逊 0.00151858676786
亿元 0.00150623627139
智能化 0.00149905425278
加强 0.00147643587031
监管 0.00147057873197
旅游 0.00146754389235
信息技术 0.00145205512329
治理 0.00145157703188
主题: 1
信用卡 0.00451270487116
变量 0.00335886571477
图形 0.00333337852386
函数 0.00256773634133
参数 0.00231625686913
通用 0.00229229238716
课程 0.00224021423885
向量 0.00215332835509
合适 0.00213349592528
概率 0.00206589947707
用途 0.00197585580746
节点 0.00197117049855
学校 0.0019475826827
案件 0.00193916087894
有没有 0.00191649830129
人脸 0.00187377925371
访问 0.00186900168441
那种 0.00185180861644
样本 0.0018336599148
回归 0.00181428764535
主题: 2
随机 0.00240705087116
截止 0.00235581799898
向量 0.00234447501178
引起 0.00232469483689
缺失 0.00229765896451
没错 0.00219704936808
高层 0.00213091313818
参数 0.00210150527491
输出 0.0020846324936

## 用HDP预测给定文章的主题分布

In [18]:
lda_vector = hdp[mm_corpus[doc_num]]
# Prints the most likely Topic. Performs Max based on the 2nd element in the tuple
print(max(lda_vector, key=lambda item: item[1]))
print(hdp.print_topic(max(lda_vector, key=lambda item: item[1])[0]))
print(documents[doc_num])

(47, 0.38146775736441951)
0.003*"经理" + 0.003*"研究所" + 0.002*"预见" + 0.002*"为例" + 0.002*"尚未" + 0.002*"360" + 0.002*"顺利" + 0.002*"宇宙" + 0.002*"两倍" + 0.002*"几年"
学 法网 APP ， 法律 人 必备 ！ （ 司考 交流 ， 法律 路上 ， 法律 求职 ） 长 按 识别 二维码 下载 学 法网 APP2016 年 国家 司法考试 已 于 今天 （ 9 月 25 日 ） 落下 帷幕 ， 学 法网 （ xuefa . com ） 迎来 了 访问 高峰 ， 学 法网 APP 的 打开 率 也 急剧 上升 （ 打开 学 法网 缓慢 的 童鞋 ， 请 耐心 等待 一下 哦 ） 。 在 学 法网 APP 上 ， 有吐槽 2016 年司 考真题 的 、 有司 考 难易度 的 投票 、 有 总结 自己 考试 经历 的 等等 。 考虑 到 大家 刚考 完 不想 看太多 文字 ， 于是 本小 编特 将 学法 网上 一些 热门 投票 以 “ 数据 ” 为 核心 ， 并 以 图片 形式 展现 给 大家 ， 以 让 大家 能 更 宏观 的 了解 2016 年 司法考试 。 一 、 2016 年司 考 各卷 难度 排行 ： 卷 四 > 卷 二 > 卷 三 > 卷 一 25 日 下午 考试 结束 以后 ， 学 法网 第一 时间 就 在 网上 对 参加 了 当天 考试 的 考生 进行 了 《 2016 年 司法考试 ， 哪卷 最难 ？ 》 的 投票 ， 被 吐 槽 最 多 的 是 卷 二 和 卷 四 （ 觉得 卷 四 最难 的 占 38.42% ， 觉得 卷二 最难 的 占 34.48% ） 、 其次 是 卷 三 ， 最后 是 卷 一 ， 数据 如下 图 ： 投票 地址 ： http : / / bbs . xuefa . com / thread - 540535 - 1 - 1 . html 二 、 2016 年 司考考 场 缺考 情况 ： 司考 人数 很多 ， 打 酱油 的 也 很多 通过 分析 学法 网上 的 投票 贴 《 2016 年司 考 ， 还记 的 您 的 考场 上 缺考 几人 吗 ？ 》 得出结论 ， 考场 缺考 人数 达到 6 人 